In [17]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2TextModelWithProjection, AutoProcessor, Blip2VisionModelWithProjection
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [20]:
model = Blip2TextModelWithProjection.from_pretrained(
    "Salesforce/blip2-itm-vit-g", device_map="auto", torch_dtype=torch.float16
)
model.to(device)
processor = AutoProcessor.from_pretrained("Salesforce/blip2-itm-vit-g")
inputs = processor(text=["a photo of a cat", "a photo of a dog", "not a photo of a dog"], return_tensors="pt", padding=True).to(device)

outputs = model(**inputs)
text_embeds = outputs.text_embeds[:,0,:]
print(text_embeds.shape)

torch.Size([3, 256])


In [12]:


model = Blip2VisionModelWithProjection.from_pretrained(
    "Salesforce/blip2-itm-vit-g", device_map="auto"
)
model.to(device)
processor = AutoProcessor.from_pretrained("Salesforce/blip2-itm-vit-g")
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
outputs = model(**inputs)
image_embeds = outputs.image_embeds
print(image_embeds.shape)


torch.Size([1, 32, 256])


In [15]:
outputs.last_hidden_state.shape

torch.Size([1, 257, 1408])